In [1]:
from ase_interface import ANIENS
from ase_interface import ensemblemolecule

import pyNeuroChem as pync
import pyaniasetools as pya
import hdnntools as hdt

import numpy as np
import  ase

import time

import os
os.environ["OMP_NUM_THREADS"] = "2"

import  ase
from ase.md.langevin import Langevin
from ase.md.verlet import VelocityVerlet
from ase.io.trajectory import Trajectory
from ase import units

from ase.optimize.fire import FIRE as QuasiNewton
from ase.optimize import LBFGS

from ase.md.nvtberendsen import NVTBerendsen
from ase.md import MDLogger

from ase.io import read, write

from ase.parallel import world

In [2]:
dir = '/home/jsmith48/scratch/carbon_nano_tube/'

# Molecule file
molfile = dir + 'mdcrd.xyz'

# Dynamics file
xyzfile = dir + 'mdcrd.xyz'

# Trajectory file
trajfile = dir + 'traj.dat'

# Optimized structure out:
optfile = dir + 'optmol_dft.xyz'

T = 300.0 # Temperature
dt = 0.25
C = 0.0007 # Optimization convergence
steps = 1000

ntdir = '/home/jsmith48/scratch/ccsd_extrapolation/final_train_models/ani-1x_dft_x8ens/'
cns = ntdir + 'rHCNO-5.0R_16-3.4A_a4-8.params'
sae = ntdir + 'sae_linfit.dat'
nnf = ntdir + 'train'
Nn = 8


#ntdir = '/home/jsmith48/scratch/ANI-1x_retrain/train_ens/'
#cns = ntdir + 'rHCNO-4.6R_16-3.1A_a4-8.params'
#sae = ntdir + 'sae_linfit.dat'
#nnf = ntdir + 'train'
#Nn = 5

In [3]:
# Load molecule
mol = read(molfile)
# L = 40.0
# mol.set_cell(([[L, 0, 0],
#                [0, L, 0],
#                [0, 0, L]]))

# mol.set_pbc((True, True, True))

# Set NC
aens = ensemblemolecule(cns, sae, nnf, Nn, 0)

# Set ANI calculator
mol.set_calculator(ANIENS(aens,sdmx=20000000.0))

s_time = time.time()
E = mol.get_potential_energy()
F = mol.get_forces()
print('Total time:',time.time()-s_time)

spc = mol.get_chemical_symbols()
print(len(spc))

Total time: 0.3155550956726074
2154


In [4]:
# Optimize molecule
start_time = time.time()
dyn = LBFGS(mol)
dyn.run(fmax=C)
print('[ANI Total time:', time.time() - start_time, 'seconds]')

# Save optimized mol
spc = mol.get_chemical_symbols()
pos = mol.get_positions(wrap=False).reshape(1,len(spc),3)


hdt.writexyzfile(optfile, pos, spc)

       Step     Time          Energy         fmax
LBFGS:    0 12:52:12 -2092614.938564        5.6254
LBFGS:    1 12:52:12 -2092639.415906        2.7409
LBFGS:    2 12:52:12 -2092650.443773        1.0081
LBFGS:    3 12:52:12 -2092656.378745        1.2262
LBFGS:    4 12:52:12 -2092658.880490        1.0215
LBFGS:    5 12:52:12 -2092660.505835        0.6232
LBFGS:    6 12:52:12 -2092661.493188        0.6810
LBFGS:    7 12:52:12 -2092662.321986        0.3961
LBFGS:    8 12:52:12 -2092662.948001        0.7328
LBFGS:    9 12:52:12 -2092663.346202        0.7954
LBFGS:   10 12:52:12 -2092663.647782        0.4829
LBFGS:   11 12:52:12 -2092663.919540        0.5397
LBFGS:   12 12:52:12 -2092664.149746        0.7549
LBFGS:   13 12:52:12 -2092664.335259        0.6078
LBFGS:   14 12:52:13 -2092664.508194        0.3059
LBFGS:   15 12:52:13 -2092664.659382        0.5224
LBFGS:   16 12:52:13 -2092664.773404        0.4807
LBFGS:   17 12:52:13 -2092664.868805        0.3272
LBFGS:   18 12:52:13 -2092664.95

LBFGS:  160 12:52:20 -2092666.048762        0.0074
LBFGS:  161 12:52:20 -2092666.048928        0.0067
LBFGS:  162 12:52:20 -2092666.049206        0.0072
LBFGS:  163 12:52:20 -2092666.049420        0.0073
LBFGS:  164 12:52:20 -2092666.049668        0.0063
LBFGS:  165 12:52:20 -2092666.049871        0.0059
LBFGS:  166 12:52:20 -2092666.050148        0.0064
LBFGS:  167 12:52:21 -2092666.050358        0.0070
LBFGS:  168 12:52:21 -2092666.050572        0.0074
LBFGS:  169 12:52:21 -2092666.050761        0.0071
LBFGS:  170 12:52:21 -2092666.051004        0.0070
LBFGS:  171 12:52:21 -2092666.051268        0.0065
LBFGS:  172 12:52:21 -2092666.051419        0.0063
LBFGS:  173 12:52:21 -2092666.051674        0.0071
LBFGS:  174 12:52:21 -2092666.051842        0.0061
LBFGS:  175 12:52:21 -2092666.052121        0.0061
LBFGS:  176 12:52:21 -2092666.052372        0.0074
LBFGS:  177 12:52:21 -2092666.052580        0.0069
LBFGS:  178 12:52:21 -2092666.052838        0.0066
LBFGS:  179 12:52:21 -2092666.0

LBFGS:  321 12:52:28 -2092666.065563        0.0022
LBFGS:  322 12:52:28 -2092666.065561        0.0020
LBFGS:  323 12:52:28 -2092666.065590        0.0023
LBFGS:  324 12:52:28 -2092666.065618        0.0026
LBFGS:  325 12:52:28 -2092666.065613        0.0024
LBFGS:  326 12:52:28 -2092666.065624        0.0022
LBFGS:  327 12:52:28 -2092666.065608        0.0024
LBFGS:  328 12:52:28 -2092666.065695        0.0021
LBFGS:  329 12:52:28 -2092666.065646        0.0023
LBFGS:  330 12:52:28 -2092666.065683        0.0021
LBFGS:  331 12:52:28 -2092666.065706        0.0018
LBFGS:  332 12:52:28 -2092666.065729        0.0021
LBFGS:  333 12:52:28 -2092666.065754        0.0020
LBFGS:  334 12:52:28 -2092666.065729        0.0027
LBFGS:  335 12:52:28 -2092666.065776        0.0019
LBFGS:  336 12:52:28 -2092666.065790        0.0020
LBFGS:  337 12:52:28 -2092666.065816        0.0020
LBFGS:  338 12:52:28 -2092666.065847        0.0021
LBFGS:  339 12:52:28 -2092666.065794        0.0024
LBFGS:  340 12:52:29 -2092666.0

LBFGS:  482 12:52:35 -2092666.067421        0.0015
LBFGS:  483 12:52:35 -2092666.067417        0.0019
LBFGS:  484 12:52:35 -2092666.067439        0.0018
LBFGS:  485 12:52:35 -2092666.067427        0.0017
LBFGS:  486 12:52:35 -2092666.067446        0.0018
LBFGS:  487 12:52:35 -2092666.067417        0.0022
LBFGS:  488 12:52:35 -2092666.067429        0.0017
LBFGS:  489 12:52:35 -2092666.067442        0.0017
LBFGS:  490 12:52:36 -2092666.067443        0.0015
LBFGS:  491 12:52:36 -2092666.067457        0.0016
LBFGS:  492 12:52:36 -2092666.067465        0.0018
LBFGS:  493 12:52:36 -2092666.067548        0.0018
LBFGS:  494 12:52:36 -2092666.067499        0.0016
LBFGS:  495 12:52:36 -2092666.067520        0.0017
LBFGS:  496 12:52:36 -2092666.067488        0.0018
LBFGS:  497 12:52:36 -2092666.067536        0.0017
LBFGS:  498 12:52:36 -2092666.067521        0.0017
LBFGS:  499 12:52:36 -2092666.067512        0.0017
LBFGS:  500 12:52:36 -2092666.067561        0.0023
LBFGS:  501 12:52:36 -2092666.0

LBFGS:  643 12:52:43 -2092666.068356        0.0019
LBFGS:  644 12:52:43 -2092666.068350        0.0016
LBFGS:  645 12:52:43 -2092666.068365        0.0022
LBFGS:  646 12:52:43 -2092666.068365        0.0016
LBFGS:  647 12:52:43 -2092666.068368        0.0018
LBFGS:  648 12:52:43 -2092666.068392        0.0017
LBFGS:  649 12:52:43 -2092666.068378        0.0018
LBFGS:  650 12:52:43 -2092666.068438        0.0017
LBFGS:  651 12:52:43 -2092666.068398        0.0019
LBFGS:  652 12:52:43 -2092666.068402        0.0018
LBFGS:  653 12:52:43 -2092666.068402        0.0017
LBFGS:  654 12:52:44 -2092666.068433        0.0019
LBFGS:  655 12:52:44 -2092666.068425        0.0020
LBFGS:  656 12:52:44 -2092666.068445        0.0017
LBFGS:  657 12:52:44 -2092666.068490        0.0018
LBFGS:  658 12:52:44 -2092666.068521        0.0022
LBFGS:  659 12:52:44 -2092666.068474        0.0018
LBFGS:  660 12:52:44 -2092666.068438        0.0015
LBFGS:  661 12:52:44 -2092666.068484        0.0020
LBFGS:  662 12:52:44 -2092666.0

...

LBFGS:  5864 12:57:01 -2092666.096984        0.0023
LBFGS:  5865 12:57:01 -2092666.097028        0.0012
LBFGS:  5866 12:57:01 -2092666.097037        0.0023
LBFGS:  5867 12:57:02 -2092666.097057        0.0010
LBFGS:  5868 12:57:02 -2092666.097019        0.0021
LBFGS:  5869 12:57:02 -2092666.097017        0.0012
LBFGS:  5870 12:57:02 -2092666.097027        0.0024
LBFGS:  5871 12:57:02 -2092666.097020        0.0010
LBFGS:  5872 12:57:02 -2092666.097020        0.0021
LBFGS:  5873 12:57:02 -2092666.097018        0.0010
LBFGS:  5874 12:57:02 -2092666.097044        0.0019
LBFGS:  5875 12:57:02 -2092666.097011        0.0010
LBFGS:  5876 12:57:02 -2092666.096979        0.0019
LBFGS:  5877 12:57:02 -2092666.097040        0.0013
LBFGS:  5878 12:57:02 -2092666.097025        0.0025
LBFGS:  5879 12:57:02 -2092666.097016        0.0013
LBFGS:  5880 12:57:02 -2092666.097000        0.0020
LBFGS:  5881 12:57:02 -2092666.097044        0.0013
LBFGS:  5882 12:57:02 -2092666.096991        0.0026
LBFGS:  5883

LBFGS:  6022 12:57:09 -2092666.097214        0.0022
LBFGS:  6023 12:57:09 -2092666.097197        0.0011
LBFGS:  6024 12:57:09 -2092666.097241        0.0017
LBFGS:  6025 12:57:09 -2092666.097256        0.0013
LBFGS:  6026 12:57:09 -2092666.097220        0.0023
LBFGS:  6027 12:57:09 -2092666.097257        0.0012
LBFGS:  6028 12:57:09 -2092666.097233        0.0019
LBFGS:  6029 12:57:10 -2092666.097190        0.0013
LBFGS:  6030 12:57:10 -2092666.097227        0.0027
LBFGS:  6031 12:57:10 -2092666.097224        0.0012
LBFGS:  6032 12:57:10 -2092666.097249        0.0021
LBFGS:  6033 12:57:10 -2092666.097224        0.0012
LBFGS:  6034 12:57:10 -2092666.097274        0.0021
LBFGS:  6035 12:57:10 -2092666.097256        0.0012
LBFGS:  6036 12:57:10 -2092666.097273        0.0020
LBFGS:  6037 12:57:10 -2092666.097282        0.0012
LBFGS:  6038 12:57:10 -2092666.097258        0.0023
LBFGS:  6039 12:57:10 -2092666.097242        0.0012
LBFGS:  6040 12:57:10 -2092666.097310        0.0023
LBFGS:  6041

LBFGS:  6180 12:57:18 -2092666.097480        0.0019
LBFGS:  6181 12:57:18 -2092666.097440        0.0012
LBFGS:  6182 12:57:18 -2092666.097423        0.0016
LBFGS:  6183 12:57:18 -2092666.097423        0.0013
LBFGS:  6184 12:57:18 -2092666.097439        0.0018
LBFGS:  6185 12:57:18 -2092666.097408        0.0012
LBFGS:  6186 12:57:18 -2092666.097414        0.0021
LBFGS:  6187 12:57:18 -2092666.097412        0.0010
LBFGS:  6188 12:57:18 -2092666.097457        0.0020
LBFGS:  6189 12:57:18 -2092666.097430        0.0011
LBFGS:  6190 12:57:18 -2092666.097439        0.0030
LBFGS:  6191 12:57:18 -2092666.097411        0.0015
LBFGS:  6192 12:57:18 -2092666.097391        0.0021
LBFGS:  6193 12:57:18 -2092666.097455        0.0014
LBFGS:  6194 12:57:19 -2092666.097463        0.0021
LBFGS:  6195 12:57:19 -2092666.097483        0.0012
LBFGS:  6196 12:57:19 -2092666.097467        0.0029
LBFGS:  6197 12:57:19 -2092666.097481        0.0012
LBFGS:  6198 12:57:19 -2092666.097502        0.0019
LBFGS:  6199

LBFGS:  6338 12:57:26 -2092666.097596        0.0021
LBFGS:  6339 12:57:26 -2092666.097607        0.0009
LBFGS:  6340 12:57:26 -2092666.097607        0.0015
LBFGS:  6341 12:57:26 -2092666.097604        0.0012
LBFGS:  6342 12:57:26 -2092666.097642        0.0020
LBFGS:  6343 12:57:26 -2092666.097677        0.0010
LBFGS:  6344 12:57:26 -2092666.097619        0.0022
LBFGS:  6345 12:57:26 -2092666.097610        0.0010
LBFGS:  6346 12:57:26 -2092666.097570        0.0022
LBFGS:  6347 12:57:26 -2092666.097582        0.0010
LBFGS:  6348 12:57:26 -2092666.097574        0.0021
LBFGS:  6349 12:57:26 -2092666.097582        0.0010
LBFGS:  6350 12:57:26 -2092666.097628        0.0022
LBFGS:  6351 12:57:26 -2092666.097608        0.0010
LBFGS:  6352 12:57:26 -2092666.097578        0.0029
LBFGS:  6353 12:57:26 -2092666.097629        0.0010
LBFGS:  6354 12:57:27 -2092666.097637        0.0018
LBFGS:  6355 12:57:27 -2092666.097635        0.0012
LBFGS:  6356 12:57:27 -2092666.097670        0.0025
LBFGS:  6357

LBFGS:  6496 12:57:34 -2092666.097732        0.0024
LBFGS:  6497 12:57:34 -2092666.097700        0.0009
LBFGS:  6498 12:57:34 -2092666.097681        0.0025
LBFGS:  6499 12:57:34 -2092666.097722        0.0008
LBFGS:  6500 12:57:34 -2092666.097688        0.0016
LBFGS:  6501 12:57:34 -2092666.097716        0.0008
LBFGS:  6502 12:57:34 -2092666.097708        0.0017
LBFGS:  6503 12:57:34 -2092666.097704        0.0008
LBFGS:  6504 12:57:34 -2092666.097704        0.0019
LBFGS:  6505 12:57:34 -2092666.097689        0.0008
LBFGS:  6506 12:57:34 -2092666.097696        0.0013
LBFGS:  6507 12:57:34 -2092666.097725        0.0008
LBFGS:  6508 12:57:34 -2092666.097677        0.0019
LBFGS:  6509 12:57:34 -2092666.097724        0.0008
LBFGS:  6510 12:57:34 -2092666.097741        0.0017
LBFGS:  6511 12:57:35 -2092666.097713        0.0007
LBFGS:  6512 12:57:35 -2092666.097710        0.0016
LBFGS:  6513 12:57:35 -2092666.097688        0.0007
LBFGS:  6514 12:57:35 -2092666.097659        0.0018
LBFGS:  6515

In [ ]:
# We want to run MD with constant energy using the VelocityVerlet algorithm.
#dyn = NVTBerendsen(mol, 1 * units.fs, 100.0, taut=1.0*1000*units.fs)
# coefficient to 0.04 atomic units.
dyn = Langevin(mol, dt * units.fs, T * units.kB, 0.04)

In [ ]:
# Open MD output
mdcrd = open(xyzfile,'w')

# Open MD output
traj = open(trajfile,'w')

# Define the printer
def storeenergy(a=mol, d=dyn, b=mdcrd, t=traj):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)

    stddev =  hdt.evtokcal*a.calc.stddev

    t.write(str(d.get_number_of_steps()) + ' ' + str(ekin / (1.5 * units.kB)) + ' ' + str(epot) + ' ' + str(ekin) + ' ' + str(epot+ekin) + ' ' + str(stddev) + '\n')
    b.write(str(len(a)) + '\n' + str(ekin / (1.5 * units.kB)) + ' Step: ' + str(d.get_number_of_steps()) + '\n')
    c = a.get_positions(wrap=False)
    for j, i in zip(a, c):
        b.write(str(j.symbol) + ' ' + str(i[0]) + ' ' + str(i[1]) + ' ' + str(i[2]) + '\n')

    print('Step: %d Energy per atom: Epot = %.3f  Ekin = %.3f (T=%.3fK)  '
          'Etot = %.6f Esig = %.6f' % (d.get_number_of_steps(), epot, ekin, ekin / (1.5 * units.kB), epot + ekin, stddev))
    
# Set the printer
dyn.attach(storeenergy, interval=10)

In [ ]:
start_time = time.time()
dyn.set_temperature(300. * units.kB)
dyn.run(steps)
print('[ANI Total time:', time.time() - start_time, 'seconds]')

In [ ]:
dyn.run(steps)